In [11]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import numpy as np 
import os 
import pickle
from tqdm.notebook import tqdm
from tensorflow import keras
from PIL import Image
import matplotlib.pyplot as plt 
from tensorflow.keras.applications.vgg16 import VGG16 , preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical , plot_model
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,Dropout, add

In [2]:
Base_dir='/content/drive/MyDrive/Image Caption Generator Project'

In [3]:
# extract features from each photo in the directory
def extract_features(filename):
    # load the model
    model = VGG16()
    # re-structure the model
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    # load the photo
    image = load_img(filename, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # get features
    feature = model.predict(image, verbose=0)
    return feature

# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text

In [10]:
# load the tokenizer
# tokenizer = load(open('tokenizer.pkl', 'rb'))
# print(type(tokenizer))
# pre-define the max sequence length (from training)
 
# load the model
max_length = 35
tokenizer = pickle.load(open('/content/drive/MyDrive/Image Caption Generator Project/tokenizer.pkl', 'rb'))

img_path = '/content/drive/MyDrive/Image Caption Generator Project/Custom_Images/download (2).jpg'

model=keras.models.load_model('/content/drive/MyDrive/Image Caption Generator Project/model_50epochs.h5')


# load and prepare the photograph
photo = extract_features(img_path)
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
image = Image.open(img_path)
plt.imshow(image)

ans=''
for i in description.split(' '):
    if(i !='startseq' and i !='endseq'):
        ans=ans + ' ' + i
print(ans) 

FileNotFoundError: ignored

In [4]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.1 MB 6.3 MB/s 
     |████████████████████████████████| 4.7 MB 47.7 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 164 kB 52.9 MB/s 
     |████████████████████████████████| 235 kB 52.3 MB/s 
     |████████████████████████████████| 181 kB 44.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 51 kB 8.1 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=96f43e91abe1a872a6a77fdfcc1d4037f288ae4e8e8ea7631740858a40332199
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [5]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 7.6 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=554f72b4385a548d5f04ea9f41609920493c399c447490c988ca669a2aedaa08
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [6]:
from pyngrok import ngrok

In [7]:
ngrok.set_auth_token("2F00ckrpsZKEGfjpIFS3XGmGZCQ_fpPYBit9uGjKMKG4sxQw")

In [16]:
%%writefile app.py

import numpy as np 
import os 
import pickle
from tqdm.notebook import tqdm
from tensorflow import keras
from PIL import Image
import matplotlib.pyplot as plt 
from tensorflow.keras.applications.vgg16 import VGG16 , preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical , plot_model
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,Dropout, add




# extract features from each photo in the directory
def extract_features(filename):
    # load the model
    model = VGG16()
    # re-structure the model
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    # load the photo
    image = load_img(filename, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # get features
    feature = model.predict(image, verbose=0)
    return feature

# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text


# load the tokenizer
# tokenizer = load(open('tokenizer.pkl', 'rb'))
# print(type(tokenizer))
# pre-define the max sequence length (from training)
 
# load the model
max_length = 35
tokenizer = pickle.load(open('/content/drive/MyDrive/Image Caption Generator Project/tokenizer.pkl', 'rb'))

# img_path = '/content/drive/MyDrive/Image Caption Generator Project/Custom_Images/download (2).jpg'

model=keras.models.load_model('/content/drive/MyDrive/Image Caption Generator Project/model_50epochs.h5')





# =================================================================================================================

import streamlit as st
import warnings
warnings.filterwarnings('ignore')

try:

    st.title("Image Caption Generator")
    data = st.file_uploader("Upload an Image ")

    if(data is None):
        st.write("No file uploaded yet :( ")
    else:
        #my_bar = st.progress(0)
        #for percent_complete in range(100):
            #time.sleep(0.02)
            #my_bar.progress(percent_complete + 1)

        #st.write(data.name)
        #st.write(data)

        temp_dir='/content'
        with open(os.path.join(temp_dir,data.name), 'wb') as f:
            f.write(data.getbuffer())
        st.success("File saved ")

        img_path=os.path.join(temp_dir,data.name)

        # load and prepare the photograph
        photo = extract_features(img_path)

        # generate description
        description = generate_desc(model, tokenizer, photo, max_length)

        image = Image.open(img_path)
        #plt.imshow(image)

        ans=''
        for i in description.split(' '):
            if(i !='startseq' and i !='endseq'):
                ans=ans + ' ' + i
        
        
        
        st.image(image) 
        
        st.subheader(ans)
        

    











except:
    pass 





Overwriting app.py


In [15]:
!nohup streamlit run app.py --server.port 80 &
#url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://7c9a-34-125-56-69.ngrok.io" -> "http://localhost:80"


In [ ]:
ngrok.connect(port = '8080')

PyngrokNgrokHTTPError: ignored